In [ ]:
!pip install -q peft transformers trl
#!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -i https://pypi.org/simple/bitsandbytes
!pip install -q torch

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install -q -U auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install -q -U langchain

!pip install -q faiss-gpu
!pip install -q langchain_community
!pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
import torch

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gemma-2b-it-Question-generation-en-sft-qlora-Borobudur")

model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gemma-2b-it-Question-generation-en-sft-qlora-Borobudur",
                                             quantization_config=quant_config,
                                             device_map={"": 0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
doc = """
Candi Borobudur dibangun oleh Dinasti Sailendra. Candi Borobudur merupakan peninggalan Buddha terbesar di dunia yang dibangun antara tahun 780-840 Masehi. Pada masa itu, Dinasti Sailendra adalah penguasa di wilayah tersebut. Candi Borobudur didirikan sebagai tempat pemujaan Buddha dan tempat ziarah. Candi ini dirancang untuk memberikan petunjuk kepada manusia agar menjauhi nafsu dunia dan mencari pencerahan serta kebijaksanaan menurut ajaran Buddha.

Candi Borobudur ditemukan oleh tentara Inggris pada tahun 1814 di bawah pimpinan Sir Thomas Stamford Raffles. Seluruh area candi berhasil dibersihkan pada tahun 1835, sehingga candi ini bisa dipelajari.

Borobudur dibangun dengan gaya arsitektur Mandala yang mencerminkan pandangan Buddha tentang alam semesta. Struktur candi ini berbentuk persegi dengan empat pintu masuk dan titik pusat berbentuk lingkaran. Jika dilihat dari luar ke dalam, candi ini terbagi menjadi dua bagian, yaitu bagian luar yang terdiri dari tiga zona yang melambangkan alam dunia dan bagian pusat yang melambangkan alam Nirwana."""

In [ ]:
from transformers import pipeline
import re

def generate_question(context):

  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

  messages = [{"role": "user", "content": (f"Berikut ini adalah konteks yang diberikan:\n\n{context}\n\n"
        "Berdasarkan konteks di atas, buatkan 5 pertanyaan singkat dan relevan. "
        "Pastikan setiap pertanyaan hanya berfokus pada informasi yang disajikan dalam konteks, menggunakan bahasa yang jelas dan mudah dipahami, dan tidak menanyakan informasi di luar konteks yang diberikan.\n"
        "Buatlah 5 pertanyaan berdasarkan konteks yang diberikan.")}]

  prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  outputs = pipe(prompt, do_sample=True, temperature=0.8, top_k=50, top_p=0.95, max_new_tokens=1000)

  generated_question = outputs[0]["generated_text"][len(prompt):]

  # Define the regular expression pattern
  pattern = r"\d+\.\s+(.+?)\?"

  # Compile the regular expression pattern
  regex = re.compile(pattern)

  # Find all matches in the question list
  matches = regex.findall(generated_question)

  question_list = []
  # Print the extracted texts
  for match in matches:
      question_list.append(match.strip())

  # Menampilkan list pertanyaan
  return question_list

#question = generate_question(doc)
#question

## Generate Answer

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "/content/drive/MyDrive/Llama3-ChatQA-1.5-8B-Borobudur"

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map={"": 0},
#         quantization_config=quant_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "nvidia/Llama3-ChatQA-1.5-8B"

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map={"": 0},
#         quantization_config=quant_config)

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
# import torch
# import transformers

# pipeline2 = transformers.pipeline(
#         "text-generation",
#         model=model_gptq,
#         tokenizer=tokenizer,
#         use_cache=True,
#         device_map="auto",
#         max_new_tokens=500,
#         temperature=0.1,
#         return_full_text=False,
#         do_sample=True,
#         top_k=10,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id,
#         pad_token_id=tokenizer.eos_token_id, )

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

In [ ]:
# llm = HuggingFacePipeline(pipeline=pipeline2)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
# from transformers import pipeline
# from langchain import HuggingFacePipeline

# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     temperature=0.2,
#     return_full_text= False,
#     do_sample = True,
#     num_return_sequences=1,
#     top_k=10,
# )

# hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer

model_id = "afrizalha/Kancil-V1-llama3-4bit"
config = AutoConfig.from_pretrained(model_id)

BNB_CONFIG = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=BNB_CONFIG,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    return_full_text= False,
    do_sample = True,
    num_return_sequences=1,
    top_k=10,
)

hf = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
def answer_question(context, questions):
  template = """
            <|user|>
            Anda adalah asisten yang membantu menjawab pertanyaan dengan SANGAT SINGKAT dan JELAS. Gunakan HANYA informasi dari konteks yang diberikan, dan JANGAN mengulang kalimat pertanyaan. Berikan jawaban dengan MAKSIMAL 2 KALIMAT, menggunakan bahasa yang jelas dan mudah dipahami.
            Berikut adalah contoh pertanyaan dan jawaban yang baik:
            Pertanyaan: Siapa yang membangun Candi Borobudur?
            Jawaban: Dinasti Sailendra.
            {context}
            USER: {question}
            <|assistant|>
            """
  answer = []

  prompt_context = PromptTemplate(input_variables=["context", "question"], template=template)

  llm_chain_context = LLMChain(prompt=prompt_context ,llm=hf)

  for count, question in enumerate(questions):
    res =  llm_chain_context({"context": context , "question": question})['text'].strip()
    print(f"==== Pertanyaan {count+1} ====")
    print(f"Pertanyaan : {question}")
    print(f"Jawaban    : {res}\n\n")
    answer.append(res)

  return answer

Selecting Question

In [ ]:
# questions_answers = list(zip(generated_question, answered_question))

def select_questions(questions_answers):
    selected = []
    for i, (question, answer) in enumerate(questions_answers, start=1):
        print(f"Pertanyaan {i}: {question}")
        include = input("Apakah Anda ingin memasukkan pertanyaan ini ke dalam list? (y/n): ")
        if include.lower() == 'y':
            selected.append((question, answer))
    return selected

# selected_questions = select_questions(questions_answers)

# print("\nPertanyaan dan jawaban terpilih:\n")
# for q, a in selected_questions:
#     print(f"Pertanyaan: {q}")
#     print(f"Jawaban: {a}")
#     print()

### Similarity Check

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/QnA_Result (2).csv")
df_question = df[["Pertanyaan"]]
df_question

,Pertanyaan
0,Apa sebab terjadinya pelestarian Borobudur sem...
1,Berapa jumlah total relief di Borobudur
2,Apa kegunaan lorong sempit yang terbentuk oleh...
3,Apa fungsi stupa besar di puncak Borobudur
4,Berapa jumlah stupa di Borobudur dan apa tujua...
5,Borobudur terletak di mana dan merupakan monum...
6,Apa saja yang ada di sekitar Borobudur
7,Studi dan pelestarian Borobudur dimulai kapan
8,Candi Borobudur dipugar berapa kali
9,Apa saja yang ada di Borobudur


In [ ]:
loader = DataFrameLoader(df_question, page_content_column="Pertanyaan").load()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="firqaaa/indo-sentence-bert-base")
db = FAISS.from_documents(loader, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def similarity_check(questions, answers):
  saved_question = []
  saved_answer = []
  threshold = 100

  print("\n\n======================")
  print(f"|   Threshold : {threshold}  |")
  print("======================\n")

  for i in range(len(questions)):
    docs_and_scores = db.similarity_search_with_score(questions[i])
    document, score = docs_and_scores[0]
    print(f"Pertanyaan ke - {i+1}")
    print(f"Pertanyaan : {questions[i]}")
    print(f"===================")

    if score > threshold:
      print(f"Similarity Score : {score}")
      print(f"Pertanyaan Disimpan....\n\n")
      saved_question.append(questions[i])
      saved_answer.append(answers[i])
    else:
      print(f"Similarity Score : {score}")
      print(f"Pertanyaan gagal Disimpan....\n\n")

  return saved_question, saved_answer

In [ ]:
def qna_pipeline(context):
    # Generate questions and answers
    generated_question = generate_question(context)
    generated_question = [text.replace("**", "") for text in generated_question]
    answered_question = answer_question(context, generated_question)

    # Select questions
    questions_answers = list(zip(generated_question, answered_question))
    selected_questions_answers = select_questions(questions_answers)

    selected_questions, selected_answers = zip(*selected_questions_answers)

    # Similarity check
    similarity_question_check = similarity_check(selected_questions, selected_answers)
    return selected_questions, answered_question, similarity_question_check

#selected_questions, answered_question, similarity_question_check = qna_pipeline(doc)

In [ ]:
similarity_question_check

#### Saved QnA

In [ ]:
new_question = [i for i in similarity_question_check[0]]
new_answer = [i for i in similarity_question_check[1]]

In [ ]:
print(new_question)
print(new_answer)

['Borobudur terletak di mana dan merupakan monumen apa', 'Apa saja yang ada di sekitar Borobudur', 'Studi dan pelestarian Borobudur dimulai kapan', 'Candi Borobudur dipugar berapa kali', 'Apa saja yang ada di Borobudur']
['Borobudur terletak di Jawa Tengah dan Yogyakarta. Borobudur merupakan monumen Buddhis terbesar di dunia.', 'Borobudur adalah monumen Buddhis terbesar di dunia. Namun, di sekitarnya juga terdapat banyak candi lain, khususnya di Jawa Tengah dan Yogyakarta.', 'Studi dan pelestarian Borobudur telah berlangsung sejak abad ke-19.', 'Candi Borobudur telah 3 kali dipugar, yaitu pada tahun 1814-1822 oleh Thomas Stamford Raffles, kedua tahun 1907-1911 oleh Theodoor van Erp dan pemugaran besar terakhir pada tahun 1973-1983 oleh Pemerintah Indonesia dan UNESCO.', 'Borobudur adalah monumen Buddhis terbesar di dunia. Namun, di sekitarnya juga terdapat banyak candi lain, khususnya di Jawa Tengah dan Yogyakarta. Studi dan pelestarian Borobudur telah berlangsung sejak abad ke-19. Can

In [ ]:
new_data = {
        "Pertanyaan": new_question,
        "Jawaban": new_answer
    }
new_df = pd.DataFrame(new_data)

df = pd.read_csv("/content/drive/MyDrive/QnA_Result (2).csv")

updated_df = pd.concat([df, new_df], ignore_index=True)

updated_df

,Unnamed: 0,Pertanyaan,Jawaban
0,0.0,Apa sebab terjadinya pelestarian Borobudur sem...,Studi dan pelestarian Borobudur telah berlangs...
1,1.0,Berapa jumlah total relief di Borobudur,"Berdasarkan informasi yang ada, Borobudur memi..."
2,2.0,Apa kegunaan lorong sempit yang terbentuk oleh...,Lorong sempit yang terbentuk oleh pagar langka...
3,3.0,Apa fungsi stupa besar di puncak Borobudur,Stupa besar di puncak Borobudur berfungsi seba...
4,4.0,Berapa jumlah stupa di Borobudur dan apa tujua...,Borobudur memiliki 72 stupa dan satu stupa bes...
5,NaN,Borobudur terletak di mana dan merupakan monum...,Borobudur terletak di Jawa Tengah dan Yogyakar...
6,NaN,Apa saja yang ada di sekitar Borobudur,Borobudur adalah monumen Buddhis terbesar di d...
7,NaN,Studi dan pelestarian Borobudur dimulai kapan,Studi dan pelestarian Borobudur telah berlangs...
8,NaN,Candi Borobudur dipugar berapa kali,"Candi Borobudur telah 3 kali dipugar, yaitu pa..."
9,NaN,Apa saja yang ada di Borobudur,Borobudur adalah monumen Buddhis terbesar di d...


In [ ]:
updated_df.to_csv("/content/drive/MyDrive/QnA_Result (2).csv", index=False)

Gradio

In [ ]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.8 MB/s 

In [ ]:
import gradio as gr

In [ ]:
# Gradio interface
def inference(contexts):
    generated_question = generate_question(contexts)
    answered_question = answer_question(contexts, generated_question)
    questions_answers = list(zip(generated_question, answered_question))
    return questions_answers

def save_selected(questions_answers, selected_texts):
    selected_questions = []
    selected_answers = []
    for text in selected_texts:
        for q, a in questions_answers:
            if f"Question: {q}, Answer: {a}" == text:
                selected_questions.append(q)
                selected_answers.append(a)
                break
    similarity_question_check = similarity_check(selected_questions, selected_answers)
    new_question = [i for i in similarity_question_check[0]]
    new_answer = [i for i in similarity_question_check[1]]
    new_data = {"Pertanyaan": new_question, "Jawaban": new_answer}
    new_df = pd.DataFrame(new_data)
    df = pd.read_csv("/content/drive/MyDrive/QnA_Result (2).csv")
    updated_df = pd.concat([df, new_df], ignore_index=True)
    updated_df.to_csv("/content/drive/MyDrive/QnA_Result (2).csv", index=False)
    print("New data added:")
    print(new_df)
    print("Updated dataset:")
    print(updated_df)

def display_questions(context):
    questions_answers = inference(context)
    choices = [f"Question: {q}, Answer: {a}" for q, a in questions_answers]
    return choices, questions_answers

def on_submit(context):
    choices, questions_answers = display_questions(context)
    return gr.update(choices=choices), questions_answers

with gr.Blocks(css=".title { font-size: 24px; font-weight: bold; color: #2E86C1; text-align: center; margin-bottom: 20px;}") as iface:
    gr.Markdown("<div class='title'>Borobudur QnA Generator</div>")
    gr.Markdown("Enter your context to generate questions and answers. Select the questions you wish to save for later use.")

    context_input = gr.Textbox(lines=7, label="Context", placeholder="Enter your context here", elem_id="context-input")

    submit_button = gr.Button("Generate Question and Answer", elem_id="submit-button")
    generated_questions = gr.CheckboxGroup(label="Generated Questions and Answers", elem_id="generated-questions")

    questions_answers_state = gr.State()
    submit_button.click(fn=on_submit, inputs=context_input, outputs=[generated_questions, questions_answers_state])

    save_button = gr.Button("Save Selected QnA", elem_id="save-button")
    #download_file = gr.File(label="Download Updated CSV file", elem_id="download-file")
    save_button.click(fn=save_selected, inputs=[questions_answers_state, generated_questions])

iface.launch(share=True, debug=True)